# PPRを動かしたい

In [255]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time
import scipy

import torch
import optuna

import model3
import dataloader

from importlib import reload

device='cpu'

In [260]:
reload(model3)
reload(dataloader)

<module 'dataloader' from '/Users/suzukitakafumi/workspace/python_env/workspace/recsys_embed_ppr/Proposed2/dataloader.py'>

# データロード

In [222]:
data_dir = '../data_luxury_5core/valid1'
dataset = dataloader.AmazonDataset(data_dir, model_name='TransE')

In [261]:
reload(model3)
embedding_dim = 128
relation_size = len(set(list(dataset.triplet_df['relation'])))
entity_size = len(dataset.entity_list)
alpha = 0.1
mu = 0.8
kappa = [0.01, 0.02, 0.03]
#lambda_ = 0.5
model = model3.PPR_TransE(embedding_dim, relation_size, entity_size, data_dir, alpha, mu, kappa)

In [265]:
batch = dataset.get_batch(64)
posi_batch, nega_batch, ppr_vec, ppr_idx= batch
h = torch.tensor(posi_batch[:, 0], dtype=torch.long, device=device)
t = torch.tensor(posi_batch[:, 1], dtype=torch.long, device=device)
r = torch.tensor(posi_batch[:, 2], dtype=torch.long, device=device)

n_h = torch.tensor(nega_batch[:, 0], dtype=torch.long, device=device)
n_t = torch.tensor(nega_batch[:, 1], dtype=torch.long, device=device)
n_r = torch.tensor(nega_batch[:, 2], dtype=torch.long, device=device)

ppr_idx

array([4406, 3555, 3108, 2428, 5163, 5221, 3942, 5282, 2416, 2087, 4041,
       3471, 4697, 4659, 4749, 1595, 4066, 3065, 2280])

In [270]:
score, vec = model.predict(h, t, r, n_h, n_t, n_r, ppr_vec, ppr_idx)

torch.Size([1581])
torch.Size([1581, 1581])
torch.Size([3819])
torch.Size([3819, 3819])
torch.Size([7])
torch.Size([7, 7])


In [274]:
torch.norm(torch.tensor(ppr_vec) - vec)

tensor(2.2878, dtype=torch.float64, grad_fn=<NormBackward0>)

In [249]:
M = model.mk_sparse_sim_mat()
batch = dataset.get_batch(128)
ppr_vec = batch[-2]
H = nx.to_scipy_sparse_matrix(model.G)
idx = batch[-1]
bias = []
for i in idx:
    tmp = np.array([0 for i in range(len(dataset.entity_list))])
    tmp[i] = 1
    bias.append(tmp[np.newaxis])
bias = np.concatenate(bias)
bias.shape

torch.Size([1581])
torch.Size([1581, 1581])
torch.Size([3819])
torch.Size([3819, 3819])
torch.Size([7])
torch.Size([7, 7])


(39, 5407)

In [227]:
M = model.mk_sparse_sim_mat()
ppr_tensor = torch.tensor(ppr_vec, dtype=torch.float, device='cpu')
vec = torch.tensor([[] for i in range(ppr_tensor.shape[0])])
pre_size = 0
for mat in M:
    size = mat.shape[0]
    tmp = torch.mm(ppr_tensor[:, pre_size:size+pre_size], mat)
    vec = torch.cat([vec, tmp], dim=1)
    pre_size = size
vec.shape

torch.Size([1581])
torch.Size([1581, 1581])
torch.Size([3819])
torch.Size([3819, 3819])
torch.Size([7])
torch.Size([7, 7])


torch.Size([39, 5407])

In [251]:
vec_sparse = mu * alpha * ppr_vec * H + (1 - mu) * bias
vec_sparse.shape
torch.tensor(vec_sparse)

tensor([[0.0000e+00, 0.0000e+00, 2.4036e-08,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 2.4036e-08,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 2.4036e-08,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 2.4036e-08,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 2.4036e-08,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 2.4036e-08,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]], dtype=torch.float64)

In [248]:
idx = batch[-1]
bias = []
for i in idx:
    tmp = np.array([0 for i in range(len(dataset.entity_list))])
    tmp[i] = 1
    bias.append(tmp[np.newaxis])
bias = np.concatenate(bias)
bias.shape

39

In [72]:
# torch.sparse
coo = torch.tensor([H.row, H.col], dtype=torch.long)
v = torch.tensor(H.data, dtype=torch.float)
H = torch.sparse.FloatTensor(coo, v, torch.Size(H.shape))

In [225]:
batch = dataset.get_batch(128)
ppr_vec = batch[-2]
batch[-2]

array([[0.00000000e+00, 0.00000000e+00, 1.98896018e-07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.98896018e-07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.98896018e-07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 1.98896018e-07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.98896018e-07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.80269614e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [90]:
ppr_vec = torch.tensor(batch[-1], dtype=torch.float)

In [73]:
# torch.sparse
tmp = scipy.sparse.coo_matrix(batch[-1])
coo = torch.tensor([tmp.row, tmp.col], dtype=torch.long)
v = torch.tensor(tmp.data, dtype=torch.float)
ppr_vec = torch.sparse.FloatTensor(coo, v, torch.Size(tmp.shape))

In [126]:
H

<5407x5407 sparse matrix of type '<class 'numpy.int64'>'
	with 21466 stored elements in Compressed Sparse Row format>

In [130]:
ppr_vec.shape

(39, 5407)

In [136]:
ppr_vec_s = scipy.sparse.csr_matrix(ppr_vec)

In [146]:
a = ppr_vec_s * H

In [147]:
a

<39x5407 sparse matrix of type '<class 'numpy.float64'>'
	with 63317 stored elements in Compressed Sparse Row format>

In [150]:
a = np.array([[1, 2]])
a = scipy.sparse.csr_matrix(a)
b = np.array([[0, 1], [1, 2]])
b = scipy.sparse.csr_matrix(b)
print(a)
print(b)

(0, 0)	1
  (0, 1)	2
  (0, 1)	1
  (1, 0)	1
  (1, 1)	2


In [153]:
c = a * b
c.todense()

matrix([[2, 5]], dtype=int64)

In [166]:
v = torch.tensor([[] for i in range(3)])
x = torch.tensor([[1.] for i in range(3)])
print(v)
print(x)

y = torch.cat([v,x], dim=1)
print(y)
print(y.shape)

tensor([], size=(3, 0))
tensor([[1.],
        [1.],
        [1.]])
tensor([[1.],
        [1.],
        [1.]])
torch.Size([3, 1])


In [182]:
x = torch.tensor([[0., 1., i ** 2 + 1] for i in range(2)])
A = torch.tensor([[0., i, 0.] for i in range(3)])
print(x)
print(A)

tensor([[0., 1., 1.],
        [0., 1., 2.]])
tensor([[0., 0., 0.],
        [0., 1., 0.],
        [0., 2., 0.]])


In [183]:
torch.mm(x, A)

tensor([[0., 3., 0.],
        [0., 5., 0.]])

In [234]:
np.array([] for i in range(2))

array(<generator object <genexpr> at 0x12c60b930>, dtype=object)